## <a name="0">Inicio</a>

- [1. Importaciones necesarias](#1)

- [2. Diccionario de IDs de los municipios del analisis](#2)

- [3. Variables Economicas (REMESAS, COMERCIO EXT e INT, )](#3)
    *   [3.1. EDA Remesas](#3.1)
    *   [3.2. EDA Comercio Exterior e Interior](#3.2)

- [4. Graficos Variables Economicas Por Municipio](#4)




## <a name="1">1. Importaciones necesarias</a>

([Ir al inicio](#0))


In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt

## <a name="2">2. Diccionario de IDs de los municipios del analisis</a>

([Ir al inicio](#0))

In [ ]:
# Diccionario de municipios con sus IDs
municipios_ids = {
    'Hermosillo': 26030,
    'Cajeme': 26018,
    'Nogales': 26043,
    'San Luis Río Colorado': 26055,
    'Navojoa': 26042,
    'Magdalena de Kino': 26036,
    'Agua Prieta': 26002,
    'Caborca': 26017,
    'Puerto Peñasco': 26048,
    'Huatabampo': 26033
}

## <a name="3">3. Variables Economicas (REMESAS, COMERCIO EXT e INT, )</a>

([Ir al inicio](#0))

In [ ]:
# ================ Variables Economicas ===================

# Lista para almacenar todos los DataFrames
remesas = []

# Loop por cada municipio
for nombre, id_mun in municipios_ids.items():
    url = f"http://www.economia.gob.mx/datamexico/api/data.jsonrecords?Municipality={id_mun}&cube=banxico_mun_income_remittances&drilldowns=Municipality,Quarter&measures=Remittance+Amount&locale=es"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json().get("data", [])
        df = pd.DataFrame(data)[["Municipality", "Quarter", "Remittance Amount"]]
        remesas.append(df)
    else:
        print(f"Error al obtener datos de {nombre}")

# Concatenar todos los DataFrames
df_remesas = pd.concat(remesas, ignore_index=True)

# Lista para almacenar todos los DataFrames
comercio_ext = []

# Loop por cada municipio
for nombre, id_mun in municipios_ids.items():
    url = f"http://www.economia.gob.mx/datamexico/api/data?Municipality={id_mun}&Product+Level=2&cube=economy_foreign_trade_mun&drilldowns=Month,Flow&measures=Trade+Value&parents=false&locale=es"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json().get("data", [])
        if data:
            df = pd.DataFrame(data)
            columnas = ["Month", "Flow", "Trade Value"]
            df = df[[col for col in columnas if col in df.columns]]
            df.insert(0, "Municipality", nombre)  # Insertar como primera columna
            comercio_ext.append(df)
        else:
            print(f"Sin datos para {nombre}")
    else:
        print(f"Error al obtener datos de {nombre} (status {response.status_code})")

# Concatenar todos los DataFrames
df_comercio_ext = pd.concat(comercio_ext, ignore_index=True)

## <a name="3.1">3.1 EDA Remesas</a>

([Ir al inicio](#0))

In [ ]:
# Imprimir las primeras cinco filas
df_remesas.head()

In [ ]:
# Veamos los tipos de datos y valores no nulos para cada columna
df_remesas.info()

In [ ]:
# Esto imprime estadísticas básicas para todas las columnas numéricas
df_remesas.describe()

## <a name="3.2">3.2 EDA Comercio exterior e interior</a>

([Ir al inicio](#0))

In [ ]:
# Imprimir las primeras cinco filas
df_comercio_ext.head()

In [ ]:
# Veamos los tipos de datos y valores no nulos para cada columna
df_comercio_ext.info()

In [ ]:
# Esto imprime estadísticas básicas para todas las columnas numéricas
df_comercio_ext.describe()

## <a name="4">4. Graficos Variables Economicas Por Municipio</a>

([Ir al inicio](#0))

In [ ]:
# ================ Grafica x 10 Remesas netas ===================

# Crear un gráfico por municipio
for municipio in df_total["Municipality"].unique():
    df_mun = df_total[df_total["Municipality"] == municipio].sort_values("Quarter")
    plt.figure(figsize=(10, 5))
    plt.plot(df_mun["Quarter"], df_mun["Remittance Amount"], marker='o')
    plt.title(f"Remesas por trimestre - {municipio}")
    plt.xlabel("Trimestre")
    plt.ylabel("Monto de remesas (MXN)")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# ================ Grafica x 10 Comercio internacional neto ===================

# Asegurar que las fechas estén ordenadas correctamente
df_comercio_ext["Month"] = pd.to_datetime(df_comercio_ext["Month"], format="%Y-%m")

# Crear un gráfico por municipio
for municipio in df_comercio_ext["Municipality"].unique():
    df_mun = df_comercio_ext[df_comercio_ext["Municipality"] == municipio]

    plt.figure(figsize=(12, 6))

    # Graficar cada tipo de flujo por separado
    for flujo in df_mun["Flow"].unique():
        df_flujo = df_mun[df_mun["Flow"] == flujo].sort_values("Month")
        plt.plot(df_flujo["Month"], df_flujo["Trade Value"], label=flujo, marker='o')

    plt.title(f"Comercio exterior mensual - {municipio}")
    plt.xlabel("Mes")
    plt.ylabel("Valor comercial (MXN)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.xticks(rotation=45)
    plt.show()